### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t

In [3]:
movies = pd.read_csv('../data/movies_clean.csv', index_col=0)
reviews = pd.read_csv('../data/reviews_clean.csv', index_col=0)

In [4]:
# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 7. 10.  8.]
 [ 6. 10.  7.]
 [ 8.  9.  8.]
 [ 8. 10. 10.]
 [ 9.  9.  9.]
 [ 8.  9.  9.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of matrix factorization.

In [5]:
n_users = ratings_mat.shape[0] # number of rows in the matrix
n_movies = ratings_mat.shape[1] # number of movies in the matrix
num_ratings = np.sum(~np.isnan(ratings_mat))# total number of ratings in the matrix

In [18]:
np.argwhere(~np.isnan(ratings_mat[0]))[:,1]

array([0, 1, 2], dtype=int64)

In [37]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.sum(~np.isnan(ratings_mat))# total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users,latent_features) # user matrix filled with random values of shape user x latent
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    errors = []

    for i in range(iters):
        # for each iteration
        # keep track of iteration and MSE
        print("Optimizaiton Statistics")
        print(f"Iterations {i} | Mean Squared Error {np.mean(errors)}")

        # update our sse
        sse_accum = 0
        errors = []
        
        # For each user-movie pair
        for user in range(n_users):
            # if the rating exists
            ratings_index = np.argwhere(~np.isnan(ratings_mat[user]))[:,1]
            for movie in ratings_index:

                actual = ratings_mat[user,movie]
                pred = np.dot(user_mat[user,:], movie_mat[:,movie])
 
                # compute the error as the actual minus the dot product of the user and movie latent features
                error = actual - pred
                # Keep track of the sum of squared errors for the matrix
                errors.append(error**2)
                sse_accum += error**2
                
                for latent_feature in range(latent_features):

                    u_old = user_mat[user,latent_feature]
                    v_old = movie_mat[latent_feature,movie]
                    # update the values in each matrix in the direction of the gradient
                    u_new = u_old + (learning_rate * 2 * (actual - pred) * v_old)
                    v_new = v_old + (learning_rate * 2 * (actual - pred) * u_old)
                    
                    user_mat[user,latent_feature] = u_new
                    movie_mat[latent_feature,movie] = v_new



        # print results
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 3 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [38]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=3, learning_rate=0.005, iters=300)
# use your function with 3 latent features, lr of 0.005 and 10 iterations

Optimizaiton Statistics
Iterations 0 | Mean Squared Error nan
Optimizaiton Statistics
Iterations 1 | Mean Squared Error 56.104916268853486
Optimizaiton Statistics
Iterations 2 | Mean Squared Error 42.577043628021165
Optimizaiton Statistics
Iterations 3 | Mean Squared Error 26.84156774955817
Optimizaiton Statistics
Iterations 4 | Mean Squared Error 13.27148245194996
Optimizaiton Statistics
Iterations 5 | Mean Squared Error 5.19280831967382
Optimizaiton Statistics
Iterations 6 | Mean Squared Error 1.875417336844535
Optimizaiton Statistics
Iterations 7 | Mean Squared Error 0.864813905167463
Optimizaiton Statistics
Iterations 8 | Mean Squared Error 0.606747160125406
Optimizaiton Statistics
Iterations 9 | Mean Squared Error 0.5425826652790792
Optimizaiton Statistics
Iterations 10 | Mean Squared Error 0.5238833379492296
Optimizaiton Statistics
Iterations 11 | Mean Squared Error 0.5162588497174927
Optimizaiton Statistics
Iterations 12 | Mean Squared Error 0.5118146990258644
Optimizaiton Stati

c:\Users\Pontello\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Pontello\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [39]:
np.dot(user_mat, movie_mat)

array([[ 7.00592144,  9.99876981,  7.99712278],
       [ 5.98837862,  9.99843689,  7.00956411],
       [ 8.01465919,  9.00290463,  7.98694604],
       [ 8.01596616, 10.00660292,  9.98220769],
       [ 8.98691171,  8.99830135,  9.01032101],
       [ 7.99038646,  8.99480478,  9.01171694]])

In [40]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 7.00592144  9.99876981  7.99712278]
 [ 5.98837862  9.99843689  7.00956411]
 [ 8.01465919  9.00290463  7.98694604]
 [ 8.01596616 10.00660292  9.98220769]
 [ 8.98691171  8.99830135  9.01032101]
 [ 7.99038646  8.99480478  9.01171694]]
[[ 7. 10.  8.]
 [ 6. 10.  7.]
 [ 8.  9.  8.]
 [ 8. 10. 10.]
 [ 9.  9.  9.]
 [ 8.  9.  9.]]


In [43]:
(ratings_mat - np.dot(user_mat, movie_mat))

matrix([[-0.00592144,  0.00123019,  0.00287722],
        [ 0.01162138,  0.00156311, -0.00956411],
        [-0.01465919, -0.00290463,  0.01305396],
        [-0.01596616, -0.00660292,  0.01779231],
        [ 0.01308829,  0.00169865, -0.01032101],
        [ 0.00961354,  0.00519522, -0.01171694]])

_**Task: compare the predicted vs actual ratings and write your findings here**_

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 3 latent features, a learning rate of 0.005, and 450 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [44]:
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10.,  8.],
        [ 6., 10.,  7.],
        [ 8.,  9.,  8.],
        [ 8., 10., 10.],
        [ 9.,  9.,  9.],
        [ 8.,  9.,  9.]])

In [57]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=3, learning_rate=0.005, iters=450)
#use your function with 3 latent features, lr of 0.005 and 450 iterations

Optimizaiton Statistics
Iterations 0 | Mean Squared Error nan
Optimizaiton Statistics
Iterations 1 | Mean Squared Error 60.525108534947975
Optimizaiton Statistics
Iterations 2 | Mean Squared Error 49.35991211354038
Optimizaiton Statistics
Iterations 3 | Mean Squared Error 34.85987873074254
Optimizaiton Statistics
Iterations 4 | Mean Squared Error 20.203971563421973
Optimizaiton Statistics
Iterations 5 | Mean Squared Error 9.458077269812378
Optimizaiton Statistics
Iterations 6 | Mean Squared Error 3.845960654143224
Optimizaiton Statistics
Iterations 7 | Mean Squared Error 1.639422711477801
Optimizaiton Statistics
Iterations 8 | Mean Squared Error 0.9053346643176033
Optimizaiton Statistics
Iterations 9 | Mean Squared Error 0.6687608652897017
Optimizaiton Statistics
Iterations 10 | Mean Squared Error 0.5866663649858603
Optimizaiton Statistics
Iterations 11 | Mean Squared Error 0.5544171076937108
Optimizaiton Statistics
Iterations 12 | Mean Squared Error 0.5399561954754543
Optimizaiton Sta

In [58]:
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is: {}".format(preds[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 7. Not bad!")

The predicted value for the missing rating is: 7.219073491466444

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 7. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 3 latent features, 0.005 learning rate, and 500 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [59]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=3, learning_rate=0.005, iters=500)
#fit to 1000 users with 3 latent features, lr of 0.005, and 500 iterations

Optimizaiton Statistics
Iterations 0 | Mean Squared Error nan
Optimizaiton Statistics
Iterations 1 | Mean Squared Error 23.182336503867628
Optimizaiton Statistics
Iterations 2 | Mean Squared Error 10.838422647052715
Optimizaiton Statistics
Iterations 3 | Mean Squared Error 7.4780787846098855
Optimizaiton Statistics
Iterations 4 | Mean Squared Error 5.803166764006205
Optimizaiton Statistics
Iterations 5 | Mean Squared Error 4.780213048000897
Optimizaiton Statistics
Iterations 6 | Mean Squared Error 4.08313117695207
Optimizaiton Statistics
Iterations 7 | Mean Squared Error 3.5732322417609206
Optimizaiton Statistics
Iterations 8 | Mean Squared Error 3.1812114642832983
Optimizaiton Statistics
Iterations 9 | Mean Squared Error 2.8686762131002954
Optimizaiton Statistics
Iterations 10 | Mean Squared Error 2.6126554024512885
Optimizaiton Statistics
Iterations 11 | Mean Squared Error 2.3985499008416524
Optimizaiton Statistics
Iterations 12 | Mean Squared Error 2.2166442840287073
Optimizaiton St

`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [60]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 14287.

The number of ratings made for user-movie pairs that didn't have ratings is 13835713


In [61]:
# Test your results against the solution
assert num_ratings == 14287, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 13835713, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
